In [1]:
import cl_graph_bert as cgm
import torch
from torch import nn
import json
from transformers import BertModel, BertConfig, BertTokenizer, AdamW
import tqdm

from torch.utils.data import Dataset, DataLoader

In [2]:
import dgl
g = dgl.load_graphs("./graphs/industrial_and_scientific_5_core_new.dgl")[0][0]

In [7]:
sampler = dgl.dataloading.NeighborSampler([5, 10], prefetch_node_feats={"Review": ["input_ids", "attention_mask", "token_type_ids"]})
numReviews = num_reviews = g.num_nodes("Review")

train_dataloader = dgl.dataloading.DataLoader(g, {"Review":torch.arange(num_reviews//2+1)}, sampler,
                batch_size=16, shuffle=True, drop_last=False, num_workers=4)

test_dataloader = dgl.dataloading.DataLoader(g, {"Review":torch.arange(num_reviews//2, num_reviews)}, sampler,
                batch_size=16, shuffle=True, drop_last=False, num_workers=4)

In [50]:
import dgl.nn as dglnn
import torch.nn.functional as F

class GraphSAGE(nn.Module):
    def __init__(self, emb_types, emb_size, hid_feats, out_feats, rel_names):
        super().__init__()
        # https://www.jianshu.com/p/767950b560c4
        embed_dict = {ntype : nn.Parameter(torch.Tensor(emb_types[ntype], emb_size))
                      for ntype in emb_types.keys()}
        for key, embed in embed_dict.items():
            nn.init.xavier_uniform_(embed)
        self.embed = nn.ParameterDict(embed_dict)
        self.conv1 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(emb_size, hid_feats)
            for rel in rel_names}, aggregate='mean')
        self.conv2 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(hid_feats, out_feats)
            for rel in rel_names}, aggregate='mean')

    def forward(self, graph):
        # inputs are features of nodes
        print({k: graph.nodes(k) for k in graph.ntypes})
        embeds = {k: self.embed[graph.nodes(k)] for k in graph.ntypes}
        print(embeds)
        h = self.conv1(graph, embeds)
        h = {k: F.relu(v) for k, v in h.items()}
        h = self.conv2(graph, h)
        return h

In [51]:
model = GraphSAGE({x: g.number_of_nodes(x) for x in g.ntypes}, 512, 256, 2, g.etypes)

In [52]:
model.embed["Brand"].shape

torch.Size([1900, 512])

In [53]:
opt = torch.optim.Adam(model.parameters())
model.train()
for epoch in range(2):
    for input_nodes, output_nodes, blocks in train_dataloader:
        logits = []
        for block in blocks:
            out = model(block)
            logits.extend(out["Review"])
        print(logits)
        print(logits.shape)
        break
    break
'''
        loss = F.cross_entropy(logits[train_mask == 1], labels[train_mask == 1].type(torch.long))
        opt.zero_grad()
        loss.backward()
        opt.step()
        print("Epoch:", epoch, "Loss:", loss.item())
'''

KeyError: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])